In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device:', device)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**2,1), 'MB')

using device: cuda
TITAN Xp
Memory usage:
Allocated: 0.0 MB


In [8]:
batch_size = 64

train_dataset = datasets.MNIST( root='./mnist_data/',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)
test_dataset = datasets.MNIST( root='./mnist_data/',
                             train = False,
                             transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader( dataset=train_dataset,
                                          batch_size = batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader( dataset=test_dataset,
                                          batch_size = batch_size,
                                          shuffle=False)
# train_loader has (x, y) where x.size()=[64, 1, 28, 28] and y.size()=[64]

In [9]:
#class NNModel(torch.nn.Module):
#    def __init__(self):
#        super(NNModel, self).__init__()
#        self.conv1 = nn.Conv2d(1, 6, 5) # input channel, output channels, and filter size
#        self.conv2 = nn.Conv2d(6, 16, 5)
#        self.fc1 = nn.Linear(256,64)
#        self.fc2 = nn.Linear(64,10)
#        
#    def forward(self, x):
#        x = F.max_pool2d( F.relu(self.conv1(x)), 2)
#        x = F.max_pool2d( F.relu(self.conv2(x)), 2)
#        x = x.view(-1, 256)
#        x = F.relu(self.fc1(x))
#        x = self.fc2(x)
#        return F.softmax(x)

class NNModel(torch.nn.Module):
    def __init__(self):
        super(NNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, padding=(2,2)) # input channel, output channels, and filter size
        self.pool1 = nn.MaxPool2d(2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, 5, padding=(2,2))
        self.pool2 = nn.MaxPool2d(2, stride=2)
        self.fc1 = nn.Linear(3136,1024)
        self.fc2 = nn.Linear(1024,10)
        
    def forward(self, x):
        x = self.pool1( F.relu(self.conv1(x)) )
        x = self.pool2( F.relu(self.conv2(x)) )
        x = x.view(-1, 3136)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x)    
    

In [10]:
model = NNModel()

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform_(m.weight)
        torch.nn.init.zeros_(m.bias)

model.apply(weights_init)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

model = model.to(device)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        
        output = model(data)
        optimizer.zero_grad()
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx%50==0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, 
                batch_idx*len(data), 
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss))

def test():
    model.eval()
    with torch.no_grad():
        test_loss=0
        correct = 0
        for data, target in train_loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            test_loss += criterion(output,target)
            pred = output.data.max(1,keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    
        test_loss /=len(train_loader.dataset)
        print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
            test_loss, correct, 
            len(train_loader.dataset),
            100. * correct / len(train_loader.dataset)))

        test_loss=0
        correct = 0
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            test_loss += criterion(output,target)
            pred = output.data.max(1,keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    
        test_loss /=len(test_loader.dataset)
        print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, 
            len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

for epoch in range(0,100):
    train(epoch)
    if epoch%10==0:
        test()
test()

C:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.302301
Train Epoch: 0 [3200/60000 (5%)]	Loss: 2.301705
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.301496
Train Epoch: 0 [9600/60000 (16%)]	Loss: 2.302037
Train Epoch: 0 [12800/60000 (21%)]	Loss: 2.301406
Train Epoch: 0 [16000/60000 (27%)]	Loss: 2.301373
Train Epoch: 0 [19200/60000 (32%)]	Loss: 2.301100
Train Epoch: 0 [22400/60000 (37%)]	Loss: 2.301125
Train Epoch: 0 [25600/60000 (43%)]	Loss: 2.299752
Train Epoch: 0 [28800/60000 (48%)]	Loss: 2.299661
Train Epoch: 0 [32000/60000 (53%)]	Loss: 2.300187
Train Epoch: 0 [35200/60000 (59%)]	Loss: 2.298459
Train Epoch: 0 [38400/60000 (64%)]	Loss: 2.297961
Train Epoch: 0 [41600/60000 (69%)]	Loss: 2.297300
Train Epoch: 0 [44800/60000 (75%)]	Loss: 2.296876
Train Epoch: 0 [48000/60000 (80%)]	Loss: 2.296302
Train Epoch: 0 [51200/60000 (85%)]	Loss: 2.294016
Train Epoch: 0 [54400/60000 (91%)]	Loss: 2.294634
Train Epoch: 0 [57600/60000 (96%)]	Loss: 2.290728
Train set: Average loss: 0.0358, Accuracy: 20603/60000 (34%

Train Epoch: 8 [38400/60000 (64%)]	Loss: 1.496883
Train Epoch: 8 [41600/60000 (69%)]	Loss: 1.519911
Train Epoch: 8 [44800/60000 (75%)]	Loss: 1.497037
Train Epoch: 8 [48000/60000 (80%)]	Loss: 1.501297
Train Epoch: 8 [51200/60000 (85%)]	Loss: 1.510168
Train Epoch: 8 [54400/60000 (91%)]	Loss: 1.496565
Train Epoch: 8 [57600/60000 (96%)]	Loss: 1.503794
Train Epoch: 9 [0/60000 (0%)]	Loss: 1.480416
Train Epoch: 9 [3200/60000 (5%)]	Loss: 1.482895
Train Epoch: 9 [6400/60000 (11%)]	Loss: 1.465660
Train Epoch: 9 [9600/60000 (16%)]	Loss: 1.532177
Train Epoch: 9 [12800/60000 (21%)]	Loss: 1.470540
Train Epoch: 9 [16000/60000 (27%)]	Loss: 1.533197
Train Epoch: 9 [19200/60000 (32%)]	Loss: 1.484116
Train Epoch: 9 [22400/60000 (37%)]	Loss: 1.487923
Train Epoch: 9 [25600/60000 (43%)]	Loss: 1.466350
Train Epoch: 9 [28800/60000 (48%)]	Loss: 1.532947
Train Epoch: 9 [32000/60000 (53%)]	Loss: 1.487392
Train Epoch: 9 [35200/60000 (59%)]	Loss: 1.553919
Train Epoch: 9 [38400/60000 (64%)]	Loss: 1.518287
Train Epo

Train Epoch: 17 [6400/60000 (11%)]	Loss: 1.497645
Train Epoch: 17 [9600/60000 (16%)]	Loss: 1.495446
Train Epoch: 17 [12800/60000 (21%)]	Loss: 1.483774
Train Epoch: 17 [16000/60000 (27%)]	Loss: 1.470116
Train Epoch: 17 [19200/60000 (32%)]	Loss: 1.482820
Train Epoch: 17 [22400/60000 (37%)]	Loss: 1.467330
Train Epoch: 17 [25600/60000 (43%)]	Loss: 1.471813
Train Epoch: 17 [28800/60000 (48%)]	Loss: 1.462311
Train Epoch: 17 [32000/60000 (53%)]	Loss: 1.493673
Train Epoch: 17 [35200/60000 (59%)]	Loss: 1.490621
Train Epoch: 17 [38400/60000 (64%)]	Loss: 1.515102
Train Epoch: 17 [41600/60000 (69%)]	Loss: 1.476598
Train Epoch: 17 [44800/60000 (75%)]	Loss: 1.502203
Train Epoch: 17 [48000/60000 (80%)]	Loss: 1.479863
Train Epoch: 17 [51200/60000 (85%)]	Loss: 1.490021
Train Epoch: 17 [54400/60000 (91%)]	Loss: 1.497628
Train Epoch: 17 [57600/60000 (96%)]	Loss: 1.504128
Train Epoch: 18 [0/60000 (0%)]	Loss: 1.466795
Train Epoch: 18 [3200/60000 (5%)]	Loss: 1.476943
Train Epoch: 18 [6400/60000 (11%)]	Loss:

Train Epoch: 25 [32000/60000 (53%)]	Loss: 1.461828
Train Epoch: 25 [35200/60000 (59%)]	Loss: 1.477031
Train Epoch: 25 [38400/60000 (64%)]	Loss: 1.464044
Train Epoch: 25 [41600/60000 (69%)]	Loss: 1.465342
Train Epoch: 25 [44800/60000 (75%)]	Loss: 1.462535
Train Epoch: 25 [48000/60000 (80%)]	Loss: 1.535260
Train Epoch: 25 [51200/60000 (85%)]	Loss: 1.467926
Train Epoch: 25 [54400/60000 (91%)]	Loss: 1.492720
Train Epoch: 25 [57600/60000 (96%)]	Loss: 1.505402
Train Epoch: 26 [0/60000 (0%)]	Loss: 1.479084
Train Epoch: 26 [3200/60000 (5%)]	Loss: 1.495711
Train Epoch: 26 [6400/60000 (11%)]	Loss: 1.461523
Train Epoch: 26 [9600/60000 (16%)]	Loss: 1.472914
Train Epoch: 26 [12800/60000 (21%)]	Loss: 1.466493
Train Epoch: 26 [16000/60000 (27%)]	Loss: 1.462795
Train Epoch: 26 [19200/60000 (32%)]	Loss: 1.480053
Train Epoch: 26 [22400/60000 (37%)]	Loss: 1.477350
Train Epoch: 26 [25600/60000 (43%)]	Loss: 1.476099
Train Epoch: 26 [28800/60000 (48%)]	Loss: 1.491756
Train Epoch: 26 [32000/60000 (53%)]	Loss

...

Train Epoch: 59 [22400/60000 (37%)]	Loss: 1.461209
Train Epoch: 59 [25600/60000 (43%)]	Loss: 1.461582
Train Epoch: 59 [28800/60000 (48%)]	Loss: 1.476088
Train Epoch: 59 [32000/60000 (53%)]	Loss: 1.461406
Train Epoch: 59 [35200/60000 (59%)]	Loss: 1.461686
Train Epoch: 59 [38400/60000 (64%)]	Loss: 1.478957
Train Epoch: 59 [41600/60000 (69%)]	Loss: 1.463057
Train Epoch: 59 [44800/60000 (75%)]	Loss: 1.462353
Train Epoch: 59 [48000/60000 (80%)]	Loss: 1.461405
Train Epoch: 59 [51200/60000 (85%)]	Loss: 1.461194
Train Epoch: 59 [54400/60000 (91%)]	Loss: 1.461251
Train Epoch: 59 [57600/60000 (96%)]	Loss: 1.462043
Train Epoch: 60 [0/60000 (0%)]	Loss: 1.461654
Train Epoch: 60 [3200/60000 (5%)]	Loss: 1.463536
Train Epoch: 60 [6400/60000 (11%)]	Loss: 1.462544
Train Epoch: 60 [9600/60000 (16%)]	Loss: 1.462548
Train Epoch: 60 [12800/60000 (21%)]	Loss: 1.479319
Train Epoch: 60 [16000/60000 (27%)]	Loss: 1.462144
Train Epoch: 60 [19200/60000 (32%)]	Loss: 1.461375
Train Epoch: 60 [22400/60000 (37%)]	Loss

Train Epoch: 67 [48000/60000 (80%)]	Loss: 1.476847
Train Epoch: 67 [51200/60000 (85%)]	Loss: 1.462532
Train Epoch: 67 [54400/60000 (91%)]	Loss: 1.475990
Train Epoch: 67 [57600/60000 (96%)]	Loss: 1.461942
Train Epoch: 68 [0/60000 (0%)]	Loss: 1.461670
Train Epoch: 68 [3200/60000 (5%)]	Loss: 1.463265
Train Epoch: 68 [6400/60000 (11%)]	Loss: 1.461353
Train Epoch: 68 [9600/60000 (16%)]	Loss: 1.461206
Train Epoch: 68 [12800/60000 (21%)]	Loss: 1.467486
Train Epoch: 68 [16000/60000 (27%)]	Loss: 1.476678
Train Epoch: 68 [19200/60000 (32%)]	Loss: 1.477232
Train Epoch: 68 [22400/60000 (37%)]	Loss: 1.463453
Train Epoch: 68 [25600/60000 (43%)]	Loss: 1.461578
Train Epoch: 68 [28800/60000 (48%)]	Loss: 1.461180
Train Epoch: 68 [32000/60000 (53%)]	Loss: 1.464489
Train Epoch: 68 [35200/60000 (59%)]	Loss: 1.462627
Train Epoch: 68 [38400/60000 (64%)]	Loss: 1.463113
Train Epoch: 68 [41600/60000 (69%)]	Loss: 1.461154
Train Epoch: 68 [44800/60000 (75%)]	Loss: 1.461370
Train Epoch: 68 [48000/60000 (80%)]	Loss

Train Epoch: 76 [12800/60000 (21%)]	Loss: 1.461327
Train Epoch: 76 [16000/60000 (27%)]	Loss: 1.476790
Train Epoch: 76 [19200/60000 (32%)]	Loss: 1.461974
Train Epoch: 76 [22400/60000 (37%)]	Loss: 1.461342
Train Epoch: 76 [25600/60000 (43%)]	Loss: 1.476166
Train Epoch: 76 [28800/60000 (48%)]	Loss: 1.476863
Train Epoch: 76 [32000/60000 (53%)]	Loss: 1.477206
Train Epoch: 76 [35200/60000 (59%)]	Loss: 1.461157
Train Epoch: 76 [38400/60000 (64%)]	Loss: 1.462685
Train Epoch: 76 [41600/60000 (69%)]	Loss: 1.461284
Train Epoch: 76 [44800/60000 (75%)]	Loss: 1.461618
Train Epoch: 76 [48000/60000 (80%)]	Loss: 1.461355
Train Epoch: 76 [51200/60000 (85%)]	Loss: 1.462120
Train Epoch: 76 [54400/60000 (91%)]	Loss: 1.461184
Train Epoch: 76 [57600/60000 (96%)]	Loss: 1.476848
Train Epoch: 77 [0/60000 (0%)]	Loss: 1.461466
Train Epoch: 77 [3200/60000 (5%)]	Loss: 1.461627
Train Epoch: 77 [6400/60000 (11%)]	Loss: 1.476811
Train Epoch: 77 [9600/60000 (16%)]	Loss: 1.461700
Train Epoch: 77 [12800/60000 (21%)]	Loss

Train Epoch: 84 [38400/60000 (64%)]	Loss: 1.461604
Train Epoch: 84 [41600/60000 (69%)]	Loss: 1.461241
Train Epoch: 84 [44800/60000 (75%)]	Loss: 1.461448
Train Epoch: 84 [48000/60000 (80%)]	Loss: 1.461817
Train Epoch: 84 [51200/60000 (85%)]	Loss: 1.476964
Train Epoch: 84 [54400/60000 (91%)]	Loss: 1.461228
Train Epoch: 84 [57600/60000 (96%)]	Loss: 1.478253
Train Epoch: 85 [0/60000 (0%)]	Loss: 1.462028
Train Epoch: 85 [3200/60000 (5%)]	Loss: 1.476964
Train Epoch: 85 [6400/60000 (11%)]	Loss: 1.461200
Train Epoch: 85 [9600/60000 (16%)]	Loss: 1.461183
Train Epoch: 85 [12800/60000 (21%)]	Loss: 1.466815
Train Epoch: 85 [16000/60000 (27%)]	Loss: 1.476838
Train Epoch: 85 [19200/60000 (32%)]	Loss: 1.461327
Train Epoch: 85 [22400/60000 (37%)]	Loss: 1.462268
Train Epoch: 85 [25600/60000 (43%)]	Loss: 1.461646
Train Epoch: 85 [28800/60000 (48%)]	Loss: 1.461203
Train Epoch: 85 [32000/60000 (53%)]	Loss: 1.461419
Train Epoch: 85 [35200/60000 (59%)]	Loss: 1.476860
Train Epoch: 85 [38400/60000 (64%)]	Loss

Train Epoch: 93 [3200/60000 (5%)]	Loss: 1.461395
Train Epoch: 93 [6400/60000 (11%)]	Loss: 1.461681
Train Epoch: 93 [9600/60000 (16%)]	Loss: 1.476597
Train Epoch: 93 [12800/60000 (21%)]	Loss: 1.461308
Train Epoch: 93 [16000/60000 (27%)]	Loss: 1.461152
Train Epoch: 93 [19200/60000 (32%)]	Loss: 1.461210
Train Epoch: 93 [22400/60000 (37%)]	Loss: 1.476875
Train Epoch: 93 [25600/60000 (43%)]	Loss: 1.461335
Train Epoch: 93 [28800/60000 (48%)]	Loss: 1.470048
Train Epoch: 93 [32000/60000 (53%)]	Loss: 1.477516
Train Epoch: 93 [35200/60000 (59%)]	Loss: 1.461575
Train Epoch: 93 [38400/60000 (64%)]	Loss: 1.461152
Train Epoch: 93 [41600/60000 (69%)]	Loss: 1.461154
Train Epoch: 93 [44800/60000 (75%)]	Loss: 1.461203
Train Epoch: 93 [48000/60000 (80%)]	Loss: 1.461262
Train Epoch: 93 [51200/60000 (85%)]	Loss: 1.461520
Train Epoch: 93 [54400/60000 (91%)]	Loss: 1.461155
Train Epoch: 93 [57600/60000 (96%)]	Loss: 1.461161
Train Epoch: 94 [0/60000 (0%)]	Loss: 1.461302
Train Epoch: 94 [3200/60000 (5%)]	Loss: 